# Lab - Regularised regression

You explored the computational advertising dataset provided by Cogo Labs as part of the descriptive analytics exercise, and should have some intution about what factors influence email open rates. It is now time to translate this intuition into a machine learning model to predict email open rates for new customers based on their browsing behaviour.



## Setup

Lets start by importing the packages we'll need and mounting our Google Drive as before. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


We'll use the `read_csv` function to read the dataset, be sure to take a look at the [documentation](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html). There are mamy optional arguments you may find useful when working on your project dataset. 

In [2]:
df = pd.read_csv('/content/drive/My Drive/ba476-test/data/cogo-all.tsv', delimiter='\t')

We have to split the data into a training and testing set. `sklearn.model_selection` offers us automated ways of doing this which we will use in the future but since this is our first time, let's do it manually. 

We create a new column called `train` which is `True` if the instance should be included in the training by using the numpy random number generator. Once we have this column, we can filter on it to create two new dataframes. 

In [3]:
df['train'] = np.random.rand(len(df)) < 0.8

df_train = df[df.train == True]
df_test = df[df.train == False]

print(df_train.shape, df_test.shape)

(230788, 18) (57510, 18)


We can refresh our memories of what goes on in the dataset by looking at the column names. 

In [4]:
df.columns

Index(['state', 'user_id', 'browser1', 'browser2', 'browser3', 'device_type1',
       'device_type2', 'device_type3', 'device_type4', 'activity_observations',
       'activity_days', 'activity_recency', 'activity_locations',
       'activity_ids', 'age', 'gender', 'p_open', 'train'],
      dtype='object')

## Training a linear regression model

Let's start by setting up a very simple model that only cares about which browser a user uses. We will create a list, `predictors1` to hold ne column names of the predictors we want to include in our model to make indexing easier.

In [7]:
predictors = ['browser1', 'browser2', 'browser3']
X1_train = df_train[predictors]
X1_test = df_test[predictors]
y_train = df_train['p_open']
y_test = df_test['p_open']

Now we can follow the same four steps as always. First, choose a model class, instantiante the model and set hyperparameter values, then fit to your data. Remember we can access model attributes, in this case the coefficients and intercepts term.  

In [ ]:
# 1. choose model class
# 2. instantiate model
# 3. fit model to data

In [8]:
from sklearn.linear_model import LinearRegression # 1. choose model class
model = LinearRegression(fit_intercept=True)      # 2. instantiate model
model.fit(X1_train, y_train)                      # 3. fit model to data

model.coef_, model.intercept_

(array([0.01675152, 0.00675408, 0.04699452]), 0.07030910586403696)

Finally, we make predictions on the training and test set and evaluate the mean squared error. Of course, there are automated functions for this, but let's do it manually so that we can make sure we understand how it works. 

First we predict on the training data:

In [ ]:
# 4a. predict on training data, evaluate

In [9]:
y_train_fit = model.predict(X1_train)              # 4a. predict on training data

mse_train = np.mean( (y_train - y_train_fit)**2 )
print(np.sqrt(mse_train), mse_train)

0.1689604790883558 0.028547643493766713


Then on the testing data:

In [ ]:
# 4b. predict on test data, evaluate

In [10]:
y_test_fit = model.predict(X1_test)                # 4b. predict on test data
mse_test = np.mean( (y_test - y_test_fit)**2 )
print(np.sqrt(mse_test), mse_test)

0.16900489275438607 0.028562653774921537


In this case our MSE is pretty similar, so it's unlikely we overfit. Is this a "good" MSE? We don't really know yet, but we can say that our open-rate predictions are, on average, off by about 17\%. 


## Ridge regression

Recall that ridge regression minimizes $\sum_i (y_i - \hat{y}_i) + \lambda \sum_{j=1}^p \beta_j^2, $ where $\lambda$ is the regularization paramter and $\beta$ the coefficients of the model. 
In the `Ridge` model, the regularisation parameter is called `alpha`.

Since we are penalising the coefficients, it is important that all the predictors are on the same scale. Here, we'll use the `StandardScalar` transform to ensure every predictor has mean  0 and standard deviation 1. Make sure you look at the [StandardScalar documentation](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). 

Note that there is a difference between standardisation and normalisation, which simply ensures all values fall between 0 and 1. 

In [13]:
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler 

scaler = StandardScaler() 
X1_std_train = scaler.fit_transform(X1_train)
X1_std_test = scaler.transform(X1_test)

print(X1_std_train[:,:].std(axis=0), X1_std_train[:,:].mean(axis=0))
print(X1_std_test[:,:].std(axis=0), X1_std_test[:,:].mean(axis=0))

[1. 1. 1.] [8.49740000e-18 8.35500697e-17 2.01505373e-17]
[0.99609547 0.99986625 0.9934828 ] [-0.00353748 -0.00233341 -0.00313461]


Notice that after standadization the test set doesnot perfectly have mean 0 and std 1, because the transformer was fit on the training set. 

Now we can instantiate and fit the model. After fitting `Ridge.coeff_` contains the coefficients. 

In [15]:
#all_alphas = np.logspace(-4, 6, 10)

ridge = Ridge(alpha=0.5)   # instantiate
ridge.fit(X1_std_train, y_train)            # fit to training data

ridge.coef_

array([0.00563458, 0.00337172, 0.01021803])

We can compute the MSE manually as before. Let's just do the test MSE this time. 

In [ ]:
# predict and evalaute on test set

In [16]:
y_test_fit_ridge = ridge.predict(X1_std_test)
mse_test_ridge = np.mean( (y_test - y_test_fit_ridge)**2)
print(np.sqrt(mse_test_ridge), mse_test_ridge)

0.16900489269225985 0.02856265375392227


Alternatively, we can do this with a pipeline. Notice that we get the same result. 

In [ ]:
from sklearn.pipeline import make_pipeline
# create a pipeline equivalent to what was fit before

In [17]:
from sklearn.pipeline import make_pipeline

pipe = make_pipeline(StandardScaler(), Ridge(alpha=0.5))
pipe.fit(X1_train, y_train)
yhat_test = pipe.predict(X1_test)
print(np.mean( (y_test - yhat_test)**2))

0.02856265375392227


## Fitting Lasso and ElasticNet
Recall that lasso minimizes $\sum_i (y_i - \hat{y}_i) + \lambda \sum_{j=1}^p |\beta_j|, $ where $\lambda$ is the regularization paramter and $\beta$ the coefficients of the model.
The `Lasso` interface is very similar to `Ridge`. Instantiate the model, fit it to your training data, make predictions and evaluate the test MSE.  


In [ ]:
# import Lasso from sklearn.linear_model

# instantiate the Lasso model

# fit to X1_std_train

# predict on the test set

# evaluate MSE on the test set

In [ ]:
from sklearn.linear_model import Lasso
lasso = Lasso(alpha=0.001)
lasso.fit(X1_std_train, y_train)
y_test_fit_lasso = lasso.predict(X1_std_test)
mse_test_lasso = np.mean( (y_test - y_test_fit_lasso)**2)
print(np.sqrt(mse_test_lasso), mse_test_lasso, lasso.coef_)

0.16958168059859374 0.028757946394643467 [0.00442118 0.00212429 0.00921021]


Elastic nets minimize $\sum_i (y_i - \hat{y}_i) + \lambda \sum_{j=1}^p (\gamma |\beta_j| + (1-\gamma) \beta_j^2), $ so the penality term is a linear combination of the ridge and lasso penalty terms. As in `Ridge` and `Lasso` the regularization parameter $\lambda$ is called `alpha`, and the $\gamma$ parameter that trades of between the two different penalty terms is called `l1_ratio`. The `l1_ratio` must be between 0 and 1. When `l1_ratio=0`, the penalty is the $\ell_2$-norm (ridge). 

When you try multiple $\gamma$/`l1_ratio` values, it is often a good idea to try several values close to 1 (where the model will behave similar to lasso), for example `[.1, .5, .7, .9, .95, .99, 1]`. 

Go through the instantiate, fit, predict, evaluate process for `ElasticNet`.

In [ ]:
# your code here

In [ ]:
from sklearn.linear_model import ElasticNet
elnet = ElasticNet(l1_ratio=0.9)
elnet.fit(X1_std_train, y_train)
y_test_fit_elnet = elnet.predict(X1_std_test)
mse_test_elnet = np.mean( (y_test - y_test_fit_elnet)**2)
print(np.sqrt(mse_test_elnet), mse_test_elnet)

0.16999396794319874 0.028897949137073282


## Categorical predictors and polynomial basis functions

All of the features we used above contained numerical values, but what if this is not the case? Now we need a way to encode categorical values to numbers. This is typically done with one-hot encoding. For example, a column `gender` with three possible values (male, female, other) will be transformed into three binary columns, one for each possible categorical value. We will use the [`OneHotEncoder`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html) for this.

Another common transformation is to use polynomial basis functions. Suppose we have predictors $X_1, X_2$. Instead of fitting the linear model $y=\beta_0 + \beta_1 X_1 + \beta_2 X_2$, we may want to fit the (still linear) $y=\beta_0 + \beta_1 X_1 + \beta_2 X_2 + \beta_3 X_1X_2 + \beta_4X_1^2 + \beta_5X_2^2.$ To do this we must add the columns $X_1\cdot X_2, X_1^2$ and $X_2$ to the data matrix. We will use [`PolynomialFeatures`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.PolynomialFeatures.html) to add these columns automatically, notice that you can pass the maximum degree when instantiating the transformer. 


The example that follows creates a `ColumnTransformer` to transform to do preprocessing on the five feature columns. The three browser columns are left unchanged and kept (if we did not specify `remainder='passthrough'` they would have been discarded); the `gender` column is transformed to two binary columns, one for each of the genders that appear in the data; and we create quadratic (degree 2 polynomial) features for the  `activity_days` column. 

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import PolynomialFeatures

# all predictors 
numerical_predictors = ['browser1', 'browser2', 'browser3']
categorical_predictors = ['gender'] #['gender', 'state']
poly_predictors = ['activity_days']
all_predictors = numerical_predictors + categorical_predictors + poly_predictors

# list of the two transformation we want to do
t = [('cat', OneHotEncoder(), categorical_predictors), 
     ('poly', PolynomialFeatures(2, include_bias=False), poly_predictors)]

# instantiate columntransformer with our transforamtions t
col_transform = ColumnTransformer(transformers=t, remainder='passthrough')

Now we can apply the transformation to our training and testing sets. Notice that we have 7 columns after transformation: 3 for the browsers; 2 for the gender one-hot encoding and 2 for `activity_days` and `activity_days`$^2$.

In [ ]:
xt_train = col_transform.fit_transform(df_train[all_predictors])
xt_test = col_transform.fit_transform(df_test[all_predictors])
xt_train.shape

(230660, 7)

We can choose and fit a model as before on this new data. 

In [ ]:
# your code here

In [ ]:
model = LinearRegression(fit_intercept=True)
model.fit(xt_train, y_train)
yhat_test = model.predict(xt_test)
mse_test = np.mean( (y_test - yhat_test)**2)
print(mse_test)

0.028583370783762215


Did the model improve? Experiment some more and see if you can improve the model by adding features, trying different transformations or tuning the regularization parameters. 

(Hint: When you're stuck, try including `state` in the categorical predictors above.)